In [1]:
path_data = '/home/longln/Source-Code/NLP-Series/Unit1-A-Neural-Probabilistic-Language-Model/DataFolder/TinyShakespeare.txt'
import torchdata.datapipes as dp
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
import re
import string
import numpy as np
import pandas as pd

from torchdata.datapipes import functional_datapipe
import torchdata.datapipes.iter as pipes

In [2]:
import spacy
eng = spacy.load("en_core_web_sm") # Load the English model to tokenize 

2023-09-18 17:19:04.138208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 17:19:04.845195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-18 17:19:05.455168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-18 17:19:05.456046: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [3]:
# lowercase
def lowering_case(text):
    return text.lower()

# removal of special characters
def remove_special_characters(text):
    text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
    return text

# removal of stopwords
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in eng.Defaults.stop_words])

# removal of punctuations
def remove_punctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# # def split word
# def split_word(text):
#     return text.split()

In [5]:
data_pipe = dp.iter.IterableWrapper([path_data])
data_pipe = dp.iter.FileOpener(data_pipe, mode='rb')
data_pipe = dp.iter.RoutedDecoder(data_pipe)
data_pipe = data_pipe.map(lambda x: x[1])
data_pipe = data_pipe.map(lowering_case)
data_pipe = data_pipe.map(remove_special_characters)
data_pipe = data_pipe.map(remove_stopwords)
data_pipe = data_pipe.map(remove_punctuations)
# data_pipe = data_pipe.map(split_word)
# data_pipe = data_pipe.map(lambda x: x[0])
# convert to iterator

In [6]:
paragraph = ''
for i in data_pipe:
    # paragraph += i
    # print(len(i))
    print(i)

citizen proceed hear speak speak speak citizen resolved die famish resolved resolved citizen know caius marcius chief enemy people know t know t citizen let kill ll corn price t verdict talking t let away away second citizen word good citizens citizen accounted poor citizens patricians good authority surfeits relieve yield superfluity wholesome guess relieved humanely think dear leanness afflicts object misery inventory particularise abundance sufferance gain let revenge pikes ere rakes gods know speak hunger bread thirst revenge second citizen proceed especially caius marcius s dog commonalty second citizen consider services country citizen content good report fort pays proud second citizen nay speak maliciously citizen unto hath famously end soft conscienced men content country mother partly proud till altitude virtue second citizen help nature account vice way covetous citizen need barren accusations hath faults surplus tire repetition shouts o city risen stay prating capitol come c

In [ ]:
len(paragraph)

In [ ]:
text = """First Citizen:
You are all resolved rather to die than to famish?"""

In [ ]:
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

In [ ]:
def getTokens(data_iter):
    """
    Function to yield tokens from an iterator. Since, our iterator contains
    tuple of sentences (source and target), `place` parameters defines for which
    index to return the tokens for. `place=0` for source and `place=1` for target
    """
    for english in data_iter:
        yield engTokenize(english)

In [ ]:
# source_vocab = build_vocab_from_iterator(
#     getTokens(data_pipe),
#     min_freq=0,
#     specials= ['<unk>'],
#     special_first=True
# )
# source_vocab.set_default_index(source_vocab['<unk>'])

In [ ]:
# source_vocab.get_itos()[249]

In [ ]:
# len(source_vocab)

In [ ]:
def getTransform(vocab):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocab=vocab),
    )
    return text_tranform

In [ ]:
some_sentence = 'hello word'
# transformed_sentence = getTransform(source_vocab)(engTokenize(some_sentence))

In [ ]:
# transformed_sentence

In [ ]:
import itertools

@functional_datapipe("rolling")
class RollingWindow(pipes.IterDataPipe):
    def __init__(self, source_dp: pipes.IterDataPipe, window_size, step=1) -> None:
        super().__init__()
        self.source_dp = source_dp
        self.window_size = window_size
        self.step = step
    
    def __iter__(self):
        it = iter(self.source_dp)
        cur = []
        while True:
            try:
                while len(cur) < self.window_size:
                    cur.append(next(it))
                yield np.array(cur)
                for _ in range(self.step):
                    if cur:
                        cur.pop(0)
                    else:
                        next(it)
            except StopIteration:
                return

In [ ]:
dp = RollingWindow(data_pipe, 5, step=2)
it = iter(dp)
list(itertools.islice(it, 4))